In [34]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os

from scipy.stats import linregress, pearsonr
from scipy.io import loadmat

import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models, Input, Model, activations
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score, confusion_matrix, roc_auc_score, precision_recall_curve, PrecisionRecallDisplay, average_precision_score

from src.mpra_tools.fasta_utils import *
from src.tf_tools.cnn_regression import MCDropout